<a href="https://colab.research.google.com/github/yumehara/hackday_2022/blob/main/notebooks/resas_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RESASから市区町村ごとの人口ピラミッドのデータを生成 (2020年度)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import requests
import pandas as pd

In [3]:
DIR_NAME = '/content/drive/MyDrive/hackday/'

In [4]:
def get_resas(url):
    resas_url = 'https://opendata.resas-portal.go.jp'
    API_KEY = 'ZnIHUbYqhylILgyEmSgRz7OcaH4vEhcQjtTJBvFN'
    response = requests.get(resas_url + url, headers={'X-API-Key': API_KEY})
    if response.status_code == requests.codes.ok:
        return response.json().get('result')
    else:
        raise Exception(f'resas error: {response.status_code} {response.text}')

In [7]:
# 都道府県コード
pref_code = get_resas('/api/v1/prefectures')
df_pref = pd.DataFrame(pref_code)
print(len(df_pref))

47


In [8]:
# 市区町村一覧
df_city = pd.DataFrame()
for index, row in df_pref.iterrows():
    city = get_resas(f'/api/v1/cities?prefCode={row["prefCode"]}')
    _tmp = pd.DataFrame(city)
    _tmp['prefName'] = row['prefName']
    df_city = pd.concat([df_city, _tmp])

print(df_city.shape)

(1922, 5)


### bigCityFlag 特別区・行政区フラグ
- 0:一般の市区町村
- 1:政令指定都市の区
- 2:政令指定都市の市
- 3:東京都23区

In [12]:
df_city = df_city[~(df_city['bigCityFlag'] == '1')]
print(df_city.shape)

(1747, 5)


In [13]:
year = 2020
df_population = pd.DataFrame()
for index, row in df_city.iterrows():
    param = f'prefCode={row["prefCode"]}&cityCode={row["cityCode"]}&yearRight={year}&yearLeft={year}'
    pop_pyramid = get_resas(f'/api/v1/population/composition/pyramid?{param}')
    if pop_pyramid is not None:
        _tmp = pd.DataFrame(pop_pyramid['yearLeft']['data'])
        _tmp['prefCode'] = row['prefCode']
        _tmp['prefName'] = row['prefName']
        _tmp['cityCode'] = row['cityCode']
        _tmp['cityName'] = row['cityName']
        _tmp['year'] = year
        df_population = pd.concat([df_population, _tmp])
    else:
        # データがない市町村
        print(row['prefCode'], row['prefName'], row['cityCode'], row['cityName'])

print(df_population.shape)

1 北海道 01695 色丹村
1 北海道 01696 国後郡泊村
1 北海道 01697 留夜別村
1 北海道 01698 留別村
1 北海道 01699 紗那村
1 北海道 01700 蘂取村
7 福島県 07201 福島市
7 福島県 07202 会津若松市
7 福島県 07203 郡山市
7 福島県 07204 いわき市
7 福島県 07205 白河市
7 福島県 07207 須賀川市
7 福島県 07208 喜多方市
7 福島県 07209 相馬市
7 福島県 07210 二本松市
7 福島県 07211 田村市
7 福島県 07212 南相馬市
7 福島県 07213 伊達市
7 福島県 07214 本宮市
7 福島県 07301 桑折町
7 福島県 07303 国見町
7 福島県 07308 川俣町
7 福島県 07322 大玉村
7 福島県 07342 鏡石町
7 福島県 07344 天栄村
7 福島県 07362 下郷町
7 福島県 07364 檜枝岐村
7 福島県 07367 只見町
7 福島県 07368 南会津町
7 福島県 07402 北塩原村
7 福島県 07405 西会津町
7 福島県 07407 磐梯町
7 福島県 07408 猪苗代町
7 福島県 07421 会津坂下町
7 福島県 07422 湯川村
7 福島県 07423 柳津町
7 福島県 07444 三島町
7 福島県 07445 金山町
7 福島県 07446 昭和村
7 福島県 07447 会津美里町
7 福島県 07461 西郷村
7 福島県 07464 泉崎村
7 福島県 07465 中島村
7 福島県 07466 矢吹町
7 福島県 07481 棚倉町
7 福島県 07482 矢祭町
7 福島県 07483 塙町
7 福島県 07484 鮫川村
7 福島県 07501 石川町
7 福島県 07502 玉川村
7 福島県 07503 平田村
7 福島県 07504 浅川町
7 福島県 07505 古殿町
7 福島県 07521 三春町
7 福島県 07522 小野町
7 福島県 07541 広野町
7 福島県 07542 楢葉町
7 福島県 07543 富岡町
7 福島県 07544 川内村
7 福島県 07545 大熊町
7 福島県 07546 双葉町
7 福島県 

In [14]:
df_population['population'] = df_population['man'] + df_population['woman']
df_population.rename(columns={'prefName':'都道府県名', 'cityName':'市区町村名', 'cityCode':'団体コード'}, inplace=True)
print(df_population.shape)

(31958, 11)


In [21]:
df_population_pivot = df_population.pivot(index=['都道府県名', '市区町村名', '団体コード'], columns='class', values='population')
df_population_pivot['人口'] = df_population_pivot.sum(axis=1)
df_population_pivot = df_population_pivot.reset_index().sort_values('団体コード')
df_population_pivot['団体コード'] = df_population_pivot['団体コード'].astype(int)
print(df_population_pivot.shape)

(1682, 23)


In [22]:
df_population_pivot 

class,都道府県名,市区町村名,団体コード,0～4歳,10～14歳,15～19歳,20～24歳,25～29歳,30～34歳,35～39歳,...,55～59歳,5～9歳,60～64歳,65～69歳,70～74歳,75～79歳,80～84歳,85～89歳,90歳～,人口
208,北海道,札幌市,1100,67504,76504,83883,99640,97964,107482,122774,...,130145,72735,121608,137026,144919,103045,78438,58302,37299,1974801
143,北海道,函館市,1202,7237,8876,9953,9544,9506,10795,12714,...,16383,8095,17124,20700,22656,16776,13339,9350,6215,249794
180,北海道,小樽市,1203,2637,3740,4450,4225,3494,3809,4619,...,7226,3083,7372,9828,11637,8786,6878,5011,3233,111129
203,北海道,旭川市,1204,10257,12518,13181,11855,12638,14037,17251,...,21057,11358,21999,26901,30026,22681,17657,12255,8261,329655
176,北海道,室蘭市,1205,2555,2763,3688,4740,3502,3481,3912,...,4683,2581,4659,6366,7557,6148,4841,3241,1832,81874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,沖縄県,久米島町,47361,351,359,314,208,262,355,362,...,549,367,596,626,455,319,276,265,265,7180
1067,沖縄県,八重瀬町,47362,2001,1837,1643,1327,1506,1892,2093,...,1914,2108,2040,2080,1532,985,976,733,621,31016
1077,沖縄県,多良間村,47375,53,57,29,52,38,41,51,...,87,58,96,87,67,58,53,33,21,1074
1091,沖縄県,竹富町,47381,260,229,45,187,278,319,332,...,208,256,262,271,178,103,101,94,116,4043


In [23]:
df_population_pivot.to_csv(os.path.join(DIR_NAME, 'intermediate/population_pyramid.csv'), index=False)